In [1]:
print("hello world")

hello world


In [2]:
from flask import Flask, request, jsonify
from konlpy.tag import Komoran
from konlpy.tag import Okt
import jpype
import os
import re

In [3]:
from flask import Flask, request, jsonify
from konlpy.tag import Okt
import jpype
import os
import re

app = Flask(__name__)

print("Starting Flask server...")
print("JAVA_HOME:", os.environ.get('JAVA_HOME'))
print("JDK_HOME:", os.environ.get('JDK_HOME'))

try:
    if not jpype.isJVMStarted():
        print("jvm 들어옴")
        classpath = os.path.abspath("C:/Users/USER/anaconda3/envs/nearcare/Lib/site-packages/konlpy/java/open-korean-text-2.1.0.jar")
        print(f"path 받음 : {classpath}")
        jpype.startJVM(jpype.getDefaultJVMPath(), "-Djava.class.path=" + classpath, "-Xmx1024m", "-Xms512m", "-Xss256k")
        print("jvm 받음")
except Exception as e:
    print(f"JVM 시작 중 오류 발생: {e}")
    import traceback
    traceback.print_exc()

try:
    okt = Okt()
    print("Okt 초기화 성공")
except Exception as e:
    print(f"Okt 초기화 중 오류 발생: {e}")
    import traceback
    traceback.print_exc()

print("Okt 실행 후")






@app.route('/', methods=['GET'])
def flask_main():
    print("플라스크 메인 화면 들어옴")
    return "flask main 화면 입니다."



@app.route('/mine-text', methods=['POST'])
def mine_text():
    print("플라스크 /mine-text명령 접속")  # 디버깅
    data = request.json
    user_input = data['input']
    print("user_input:", user_input)  # 디버깅

    # 텍스트 마이닝으로 핵심 정보 추출
    try:
        tokens = okt.pos(user_input)
    except Exception as e:
        print(f"Komoran 처리 중 오류 발생: {e}")
        return jsonify({"error": "Komoran 처리 중 오류 발생"}), 500

    key_info = {
        "age": None,
        "gender": None,
        "disease": None,
        "location": None,
        "additional_info": []
    }

    for word, pos in tokens:
        if re.match(r'\d+세', word):
            key_info["age"] = word
        elif re.match(r'\d+살', word):
            key_info["age"] = word
        elif word in ["남성", "여성", "남자", "여자"]:
            key_info["gender"] = word
        elif "환자" in word or "증세" in word:
            key_info["disease"] = word
        elif word in ["광주", "서울", "부산", "대구", "인천", "대전", "전남", "전북", "충남", "충북", "경남", "경북", "강원도", "제주도", "전라남도", "전라북도", "충청북도", "충청남도"]:
            key_info["location"] = word
        else:
            key_info["additional_info"].append(word)

    key_info["additional_info"] = ' '.join([word for word, pos in tokens if pos not in ['Josa', 'Punctuation']])

    print("age:", key_info["age"], ", gender:", key_info["gender"], ", disease:", key_info["disease"], ", location:", key_info["location"], "additional_info:", key_info["additional_info"])
    return jsonify(key_info)

if __name__ == '__main__':
    app.run(port=5000, debug=True)


Starting Flask server...
JAVA_HOME: C:\Program Files\Java\jdk-1.8
JDK_HOME: C:\Program Files\Java\jdk-1.8
jvm 들어옴
path 받음 : C:\Users\USER\anaconda3\envs\nearcare\Lib\site-packages\konlpy\java\open-korean-text-2.1.0.jar
jvm 받음
Komoran 초기화 중 오류 발생: Java package 'kr.lucypark.okt' is not valid
Okt 실행 후
 * Serving Flask app '__main__'
 * Debug mode: on


Traceback (most recent call last):
  File "C:\Users\USER\AppData\Local\Temp\ipykernel_4804\169735109.py", line 28, in <module>
    okt = Okt()
          ^^^^^
  File "c:\Users\USER\anaconda3\envs\nearcare\Lib\site-packages\konlpy\tag\_okt.py", line 54, in __init__
    OktInterfaceJavaClass = oktJavaPackage.OktInterface
                            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Java package 'kr.lucypark.okt' is not valid
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\USER\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
